# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 10:35AM,263473,19,N18092,"NAPP Technologies, LLC",Released
1,May 23 2023 10:34AM,263472,20,9508505,VITABIOTICS USA INC.,Released
2,May 23 2023 10:24AM,263469,19,9504596,"Snap Medical Industries, LLC",Released
3,May 23 2023 10:24AM,263469,19,9504597,"Snap Medical Industries, LLC",Released
4,May 23 2023 10:17AM,263467,21,9508484,"NBTY Global, Inc.",Executing
5,May 23 2023 10:07AM,263466,15,9508490,"Emersa Waterbox, LLC",Released
6,May 23 2023 10:03AM,263465,10,9484332,Eywa Pharma Inc.,Released
7,May 23 2023 10:03AM,263465,10,9507905,Eywa Pharma Inc.,Released
8,May 23 2023 10:03AM,263465,10,9507941,Eywa Pharma Inc.,Released
9,May 23 2023 9:47AM,263302,15,ALIFBA1762RTV65,"Alliance Pharma, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,263466,Released,1
31,263467,Executing,1
32,263469,Released,2
33,263472,Released,1
34,263473,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263466,NaN,NaN,1.0
263467,NaN,1.0,NaN
263469,NaN,NaN,2.0
263472,NaN,NaN,1.0
263473,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0.0,1.0,2.0
263434,7.0,11.0,2.0
263439,0.0,16.0,7.0
263441,0.0,1.0,0.0
263443,3.0,2.0,25.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0,1,2
263434,7,11,2
263439,0,16,7
263441,0,1,0
263443,3,2,25


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,0,1,2
1,263434,7,11,2
2,263439,0,16,7
3,263441,0,1,0
4,263443,3,2,25


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,,1,2
1,263434,7,11,2
2,263439,,16,7
3,263441,,1,
4,263443,3,2,25


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC"
1,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.
2,May 23 2023 10:24AM,263469,19,"Snap Medical Industries, LLC"
4,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc."
5,May 23 2023 10:07AM,263466,15,"Emersa Waterbox, LLC"
6,May 23 2023 10:03AM,263465,10,Eywa Pharma Inc.
9,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc."
12,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated"
13,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc."
14,May 23 2023 9:32AM,263461,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",,,1
1,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,,,1
2,May 23 2023 10:24AM,263469,19,"Snap Medical Industries, LLC",,,2
3,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",,1,
4,May 23 2023 10:07AM,263466,15,"Emersa Waterbox, LLC",,,1
5,May 23 2023 10:03AM,263465,10,Eywa Pharma Inc.,,,3
6,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",,1,2
7,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",,,1
8,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",,,1
9,May 23 2023 9:32AM,263461,10,Emerginnova,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",1,,
1,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,1,,
2,May 23 2023 10:24AM,263469,19,"Snap Medical Industries, LLC",2,,
3,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",,1,
4,May 23 2023 10:07AM,263466,15,"Emersa Waterbox, LLC",1,,
5,May 23 2023 10:03AM,263465,10,Eywa Pharma Inc.,3,,
6,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",2,1,
7,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",1,,
8,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",1,,
9,May 23 2023 9:32AM,263461,10,Emerginnova,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",1,,
1,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,1,,
2,May 23 2023 10:24AM,263469,19,"Snap Medical Industries, LLC",2,,
3,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",,1,
4,May 23 2023 10:07AM,263466,15,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",1.0,NaN,NaN
1,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,1.0,NaN,NaN
2,May 23 2023 10:24AM,263469,19,"Snap Medical Industries, LLC",2.0,NaN,NaN
3,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,May 23 2023 10:07AM,263466,15,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",1.0,0.0,0.0
1,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,1.0,0.0,0.0
2,May 23 2023 10:24AM,263469,19,"Snap Medical Industries, LLC",2.0,0.0,0.0
3,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,May 23 2023 10:07AM,263466,15,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2898026,30.0,11.0,0.0
15,526768,3.0,1.0,0.0
19,1844169,49.0,51.0,3.0
20,1053824,13.0,20.0,7.0
21,263467,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2898026,30.0,11.0,0.0
1,15,526768,3.0,1.0,0.0
2,19,1844169,49.0,51.0,3.0
3,20,1053824,13.0,20.0,7.0
4,21,263467,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,30.0,11.0,0.0
1,15,3.0,1.0,0.0
2,19,49.0,51.0,3.0
3,20,13.0,20.0,7.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,15,Released,3.0
2,19,Released,49.0
3,20,Released,13.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,0.0,3.0,7.0,0.0
Executing,11.0,1.0,51.0,20.0,1.0
Released,30.0,3.0,49.0,13.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,0.0,3.0,7.0,0.0
1,Executing,11.0,1.0,51.0,20.0,1.0
2,Released,30.0,3.0,49.0,13.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,0.0,3.0,7.0,0.0
1,Executing,11.0,1.0,51.0,20.0,1.0
2,Released,30.0,3.0,49.0,13.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()